<a href="https://colab.research.google.com/github/anthony-rio/Portfolio/blob/main/CFBD_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load Data

In [459]:
import pandas as pd
import numpy as np
full_merged_data = pd.read_csv('full_merged_data.csv')
new_data = pd.read_csv('new_data.csv')

full_data = full_merged_data
full_data = full_data[~((full_data['Week'] == 7) & (full_data['Season'] == 2024))]
full_data = pd.concat([full_data, new_data], ignore_index=True)
# exclude_teams = ['App State', 'Jacksonville State', 'Kennesaw State', 'Massachusetts', 'Sam Houston', 'Southern Miss', 'UConn', 'UL Monroe', 'UTSA']

# Filter the DataFrame
filtered_data = full_data[full_data['Week'] != 1]
this_week = 8


filtered_data['SOS'] = pd.to_numeric(filtered_data['SOS'], errors='coerce')
filtered_data['SOS_Away'] = pd.to_numeric(filtered_data['SOS_Away'], errors='coerce')
this_week_data = filtered_data[(filtered_data['Week'] == this_week) & (filtered_data['Season'] == 2024)]
last_week_data = filtered_data[(filtered_data['Week'] == this_week - 1) & (filtered_data['Season'] == 2024)]
last_week_data = last_week_data.dropna()
# filtered_data = filtered_data[~((filtered_data['Week'] == this_week) & (filtered_data['Season'] == 2024))]
# filtered_data = filtered_data[~((filtered_data['Week'] == this_week - 1) & (filtered_data['Season'] == 2024))]
# take out 2020 COVID year and NA rows
# filtered_data = filtered_data[~(filtered_data['Season'] == 2020)]
filtered_data = filtered_data.dropna()

<ipython-input-459-f03fbbaf7580>:3: DtypeWarning: Columns (18,96) have mixed types. Specify dtype option on import or set low_memory=False.
  full_merged_data = pd.read_csv('full_merged_data.csv')
<ipython-input-459-f03fbbaf7580>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['SOS'] = pd.to_numeric(filtered_data['SOS'], errors='coerce')
<ipython-input-459-f03fbbaf7580>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['SOS_Away'] = pd.to_numeric(filtered_data['SOS_Away'], erro

Split Data

In [461]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

current_week_no_lines = this_week_data.iloc[:, 1:]
na_indices = current_week_no_lines.index[current_week_no_lines.isna().any(axis=1)].tolist()
this_week_data = this_week_data.drop(na_indices)
this_week_X = this_week_data.drop(columns=['Point Diff', 'Line', 'Home Team', 'Away Team'])
this_week_teams = this_week_data[['Home Team', 'Away Team']]
this_week_line = this_week_data['Line']

last_week_X = last_week_data.drop(columns=['Point Diff', 'Line', 'Home Team', 'Away Team'])
last_week_teams = last_week_data[['Home Team', 'Away Team']]
last_week_line = last_week_data['Line']
last_week_y = last_week_data['Point Diff']


y = filtered_data[['Point Diff', 'Line']]
X = filtered_data.drop(columns=['Point Diff', 'Line', 'Home Team', 'Away Team'])

# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=99)

# Now, split the temporary set into validation (50% of temp, which is 20% of total) and test (50% of temp, which is 20% of total)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=99)

# Extracting 'Line' for train, validation, and test sets
train_lines = y_train['Line']
val_lines = np.array(y_val['Line'])
test_lines = np.array(y_test['Line'])

# Dropping 'Line' column from y_train, y_val, and y_test
y_train = y_train.drop(columns=['Line'])
y_val = y_val.drop(columns=['Line'])
y_test = y_test.drop(columns=['Line'])

Convert to tensors for NN model

In [462]:
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)
# this_week_X = scaler.transform(this_week_X)
# last_week_X = scaler.transform(last_week_X)

# X_train = torch.tensor(np.array(X_train), dtype=torch.float32)
# X_test = torch.tensor(np.array(X_test), dtype=torch.float32)
# y_train = torch.tensor(np.array(y_train), dtype=torch.float32)
# y_test = torch.tensor(np.array(y_test), dtype=torch.float32)
# this_week_X = torch.tensor(np.array(this_week_X), dtype=torch.float32)
# this_week_line = torch.tensor(np.array(this_week_line), dtype=torch.float32)
# last_week_X = torch.tensor(np.array(last_week_X), dtype=torch.float32)
# last_week_y = torch.tensor(np.array(last_week_y), dtype=torch.float32)
# last_week_line = torch.tensor(np.array(last_week_line), dtype=torch.float32)

No Validation Set Convert to Tensors

In [463]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=80)
train_lines = y_train['Line']
test_lines = y_test['Line']
y_train = y_train.drop(columns=['Line'])
y_test = y_test.drop(columns=['Line'])
last_week_y = torch.tensor(np.array(last_week_y), dtype=torch.float32)
last_week_X = torch.tensor(np.array(last_week_X), dtype=torch.float32)
this_week_X = torch.tensor(np.array(this_week_X), dtype=torch.float32)
X_train = torch.tensor(np.array(X_train), dtype=torch.float32)
X_test = torch.tensor(np.array(X_test), dtype=torch.float32)
y_train = torch.tensor(np.array(y_train), dtype=torch.float32)
y_test = torch.tensor(np.array(y_test), dtype=torch.float32)

Better NN Model

In [464]:
class ImprovedNeuralNet(nn.Module):
    def __init__(self, input_size):
        super(ImprovedNeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)  # Increased width
        self.bn1 = nn.BatchNorm1d(256)          # Batch normalization
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)          # Batch normalization
        self.fc3 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)          # Dropout layer

    def forward(self, x):
        x = torch.relu(self.bn1(self.fc1(x)))  # Batch norm and ReLU
        x = self.dropout(x)                     # Apply dropout
        x = torch.relu(self.bn2(self.fc2(x)))  # Batch norm and ReLU
        x = self.fc3(x)                         # Output layer
        return x

# Model training with early stopping, learning rate scheduling, etc.
# Follow similar procedures as before but include enhancements as needed

# Initialize the model
input_size = X_train.shape[1]  # 165 features
nn_model = ImprovedNeuralNet(input_size)

# Define loss function and optimizer
criterion = nn.MSELoss()  # For regression
optimizer = optim.Adam(nn_model.parameters(), lr=0.001)

# Training the neural network
n_epochs = 1000
for epoch in range(n_epochs):
    nn_model.train()
    optimizer.zero_grad()
    outputs = nn_model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 50 == 0:
        print(f'Epoch [{epoch + 1}/{n_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the test set
nn_model.eval()
with torch.no_grad():
    nn_predictions = nn_model(X_test)
    test_loss = criterion(nn_predictions, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

# Optional: Print some predictions vs actual values
print("Sample Predictions vs Actual:")
for i in range(5):
    print(f'Predicted: {nn_predictions[i].item():.2f}, Actual: {y_test[i].item():.2f}')

Epoch [50/1000], Loss: 263.6720
Epoch [100/1000], Loss: 177.5360
Epoch [150/1000], Loss: 139.6231
Epoch [200/1000], Loss: 122.1205
Epoch [250/1000], Loss: 105.1701
Epoch [300/1000], Loss: 99.2404
Epoch [350/1000], Loss: 88.7924
Epoch [400/1000], Loss: 80.8468
Epoch [450/1000], Loss: 78.2105
Epoch [500/1000], Loss: 70.8695
Epoch [550/1000], Loss: 65.3569
Epoch [600/1000], Loss: 67.1062
Epoch [650/1000], Loss: 57.4642
Epoch [700/1000], Loss: 56.7580
Epoch [750/1000], Loss: 54.7585
Epoch [800/1000], Loss: 53.9848
Epoch [850/1000], Loss: 49.8485
Epoch [900/1000], Loss: 44.0080
Epoch [950/1000], Loss: 45.8777
Epoch [1000/1000], Loss: 42.4407
Test Loss: 202.7633
Sample Predictions vs Actual:
Predicted: -21.58, Actual: -2.00
Predicted: 14.71, Actual: 10.00
Predicted: -17.98, Actual: -31.00
Predicted: -32.07, Actual: -38.00
Predicted: 23.89, Actual: 28.00


Linear Regression

In [465]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the linear regression model
model = LinearRegression()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test set
lr_predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, lr_predictions)
r2 = r2_score(y_test, lr_predictions)


Random Forest

In [466]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model
rf_model.fit(X_train, y_train)

# Make predictions
rf_predictions = rf_model.predict(X_test)

# Calculate the mean squared error
mse_rf = mean_squared_error(y_test, rf_predictions)
print(f'Random Forest Mean Squared Error: {mse_rf:.4f}')

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Random Forest Mean Squared Error: 179.7716


XG Boost Model

In [467]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set parameters for XGBoost
params = {
    'objective': 'reg:squarederror',  # Regression with squared error
    'max_depth': 6,                    # Maximum depth of a tree
    'learning_rate': 0.1,              # Step size shrinkage
    'n_estimators': 100,               # Number of trees
    'random_state': 42
}

# Train the model
xgb_model = xgb.train(params, dtrain, num_boost_round=100)

# Make predictions
xgb_predictions = xgb_model.predict(dtest)

# Calculate the mean squared error
mse_xgb = mean_squared_error(y_test, xgb_predictions)
print(f'XGBoost Mean Squared Error: {mse_xgb:.4f}')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [00:08:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Mean Squared Error: 187.5582


Test Accuracy

In [449]:
# predictions = nn_model(X_test)
# predictions = rf_model.predict(X_test)
# predictions = model.predict(X_test)
# predictions = xgb_model.predict(dtest)

In [468]:
all_preds = [nn_predictions, lr_predictions, rf_predictions, xgb_predictions]
test_lines = np.array(test_lines)
accuracy = []
for predictions in all_preds:
  accuracy_temp = []
  for i in range(len(predictions)):
    if abs(predictions[i] - test_lines[i]) >= 14:
      if (predictions[i] > test_lines[i] and y_test[i] > test_lines[i]) or (predictions[i] < test_lines[i] and y_test[i] < test_lines[i]):
        accuracy_temp.append(1)
      else:
        accuracy_temp.append(0)

  accuracy.append(sum(accuracy_temp) / len(accuracy_temp))
df = pd.DataFrame({
    'Method': ['NN', 'LR', 'RF', 'XGB'],
    'Accuracy': accuracy
})
df

,Method,Accuracy
0,NN,0.757764
1,LR,0.845304
2,RF,0.881081
3,XGB,0.791519


For Instances Where The Prediction Differs By The Line By 14+ Points:

1. Neural Network (86%)
2. Logistic Regression (85%)
3. NN and RF both were just above 82% accuracy

Same Instance but no Validation Set:
1. RF: 86% Accuracy
2. LR: 83% Accuracy
3. XGB, NN: Both 81% Accuracy

70% Train Data:

- LR: 86%, NN: 82%,






In [469]:
# predictions = nn_model(X_val)
# predictions = rf_model.predict(X_val)
# valtest = xgb.DMatrix(X_val, label=y_val)
# predictions = xgb_model.predict(valtest)

In [452]:
# val_lines = np.array(val_lines)
# accuracy = []
# for i in range(len(predictions)):
  # if abs(predictions[i] - val_lines[i]) >= 14:
    # if (predictions[i] > val_lines[i] and y_val[i] > val_lines[i]) or (predictions[i] < val_lines[i] and y_val[i] < val_lines[i]):
      # accuracy.append(1)
    # else:
      # accuracy.append(0)

# accuracy = sum(accuracy) / len(accuracy)
# accuracy

Validation Set Accuracy:
1. Random Forest: 94% Accuracy
2. Linear Regression: 90% Accuracy
3. XG Boost Model: 89% Accuracy
4. Neural Net: 77% Accuracy


Test Last Week

Accuracy for 13 point differential

In [470]:
prediction_1 = nn_model(last_week_X)
xg_last_week_X = xgb.DMatrix(last_week_X, label=last_week_y)
prediction_2 = model.predict(last_week_X)
prediction_3 = rf_model.predict(last_week_X)
prediction_4 = xgb_model.predict(xg_last_week_X)



In [471]:
all_preds = [prediction_1, prediction_2, prediction_3, prediction_4]
last_week_line = np.array(last_week_line)
accuracy = []
for predictions in all_preds:
  accuracy_temp = []
  for i in range(len(predictions)):
    if abs(predictions[i] - last_week_line[i]) >= 14:
      if (predictions[i] > last_week_line[i] and last_week_y[i] > last_week_line[i]) or (predictions[i] < last_week_line[i] and last_week_y[i] < last_week_line[i]):
        accuracy_temp.append(1)
      else:
        accuracy_temp.append(0)

  accuracy.append(sum(accuracy_temp) / len(accuracy_temp))
df = pd.DataFrame({
    'Method': ['NN', 'LR', 'RF', 'XGB'],
    'Accuracy': accuracy
})
df

,Method,Accuracy
0,NN,0.888889
1,LR,1.000000
2,RF,0.857143
3,XGB,0.833333


Last Week Results No Validation set (13+ difference):
1. RF: 3/3
2. XGB: 4/5
3. LR: 5/7
4. NN: 7/9

With Val (14+):
NN: 6/9, XGB: 6/7, LR: 5/6, RF: 6/6



In [455]:
# Create a boolean mask for correct predictions
# last_week_correct = ((last_week_predictions > last_week_line) & (last_week_y > last_week_line)) | ((last_week_predictions < last_week_line) & (last_week_y < last_week_line))

# Calculate accuracy
# last_week_accuracy = np.mean(last_week_correct.astype(int))  # Convert boolean to int

# print(f"Last Week Accuracy: {last_week_accuracy:.2f}")

This past week

In [473]:
predictions = model.predict(last_week_X)
week_7_bets = pd.DataFrame({
    'Home Team': last_week_teams['Home Team'],
    'Away Team': last_week_teams['Away Team'],
    'Line': last_week_line.tolist(),
    'Prediction': predictions.squeeze().tolist(),
    'Point Diff': last_week_y.tolist()

})
week_7_bets['Bet Team'] = np.where(
    week_7_bets['Prediction'] > week_7_bets['Line'],
    week_7_bets['Away Team'],
    week_7_bets['Home Team']
)


week_7_bets[abs(week_7_bets['Prediction'] - week_7_bets['Line']) >= 14]

,Home Team,Away Team,Line,Prediction,Point Diff,Bet Team
6628,Notre Dame,Stanford,-22.5,-48.014771,-42.0,Notre Dame
6634,James Madison,Coastal Carolina,-7.0,-28.272797,-32.0,James Madison
6650,Pittsburgh,California,-3.5,10.595184,-2.0,California
6656,Maryland,Northwestern,-11.0,13.717194,27.0,Northwestern
6658,Illinois,Purdue,-23.0,-8.642670,-1.0,Purdue
6660,Oregon,Ohio State,3.5,-12.655121,-1.0,Oregon


this week test

In [474]:
#predictions = rf_model.predict(this_week_X)
predictions = model.predict(this_week_X)
# xg_this_week_X = xgb.DMatrix(this_week_X)
# predictions = xgb_model.predict(xg_this_week_X)
# predictions = nn_model(this_week_X)

In [475]:
# predictions = model.predict(last_week_X)
week_8_bets = pd.DataFrame({
    'Home Team': this_week_teams['Home Team'],
    'Away Team': this_week_teams['Away Team'],
    'Line': this_week_line.tolist(),
    'Prediction': predictions.squeeze().tolist()

})
week_8_bets['Bet Team'] = np.where(
    week_8_bets['Prediction'] > week_8_bets['Line'],
    week_8_bets['Away Team'],
    week_8_bets['Home Team']
)

print("Week 8 NN Bets")
week_8_bets[abs(week_8_bets['Prediction'] - week_8_bets['Line']) >= 14]

Week 8 NN Bets


,Home Team,Away Team,Line,Prediction,Bet Team
6694,Georgia Southern,James Madison,10.0,25.326385,James Madison
6716,Indiana,Nebraska,-6.5,9.352692,Nebraska
6720,Illinois,Michigan,3.0,26.069061,Michigan
6729,Air Force,Colorado State,7.0,-11.731262,Air Force
6731,Arkansas,LSU,2.5,19.427063,LSU
6738,Tennessee,Alabama,3.0,-12.696228,Tennessee
